In [1]:
# LangChain 接入Yuan模型
from modelscope import snapshot_download
model_dir = snapshot_download('IEITYuan/Yuan2-2B-Mars-hf', cache_dir='./model/autodl-fs')

In [1]:
# 调用模型
from LLM import Yuan2_LLM
llm = Yuan2_LLM('model/autodl-fs/IEITYuan/Yuan2-2B-Mars-hf')
print(llm("你是谁"))

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


Creat tokenizer...
Creat model...


d:\Anaconda3\envs\pytorch\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 我是一个人工智能助手，专门设计来帮助您解决问题和提供信息。无论您有什么需要，只要告诉我，我就会尽力帮助您。<eod>


In [7]:
from langchain_community.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("微课\Task3\成人高血压食养指南.pdf")
documents = loader.load()



page_content=' 
 
  
 
 
成人高血压食养指南  
（2023年版） 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
国家卫生健康委办公厅  ' metadata={'source': '微课\\Task3\\成人高血压食养指南.pdf', 'page': 0}
page_content=' 
 
  
目  录 
一、前言  ................................ .........  1 
二、疾病特点与分型  ...............................  2 
（一）高血压定义与分类  ........................  2 
（二）中医对高血压的认识及分型  ................  2 
三、食养原则和建议  ...............................  4 
（一）减钠增钾，饮食清淡  ......................  4 
（二）合理膳食，科学食养  ......................  5 
（三）吃动平衡，健康体重  ......................  6 
（四）戒烟限酒，心理平衡  ......................  7 
（五）监测血压，自我管理  ......................  7 
附录1 成人高血压患者的食物选择  ...................  9 
附录2 常见食物交换表  ............................  12 
附录3 不同地区食谱示例  ..........................  12 
一、  东北地区  ................................ ... 17 
二、西北地区  ................................ .... 22 
三、华北地区  ................................ .... 27 
四、  华东地区  ................................ ... 32 
五、  华中地区  ................................ ... 37 
六

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# 文档分割
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,length_function=len)
docs = text_splitter.split_documents(documents)

In [ ]:
from sentence_transformers import SentenceTransformer

# 指定模型名称
model_name = "moka-ai/m3e-base"

# 创建模型实例并将模型保存到指定路径
model = SentenceTransformer(model_name)
model.save('model\\m3e-base')

In [41]:
# 向量化
from transformers.utils import is_torch_cuda_available, is_torch_mps_available
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
# 词嵌入模型
EMBEDDING_DEVICE = "cuda" if is_torch_cuda_available() else "mps" if is_torch_mps_available() else "cpu"
embeddings_model = HuggingFaceEmbeddings(model_name='model\m3e-base', model_kwargs={'device': 'cpu'})

vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings_model)

In [42]:
print(vectorstore.index)
print(vectorstore.index.ntotal)  # 向量总数
print(vectorstore.index.d)       # 向量维度

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001C3F929F2A0> >
58
768


In [ ]:
# 查看元数据
metadata_list = vectorstore.docstore._dict.values()
for metadata in metadata_list:
    print(metadata)

In [60]:
# 向量查询
query = '东方健康膳食模式'
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

8 
 1.得舒饮食（ DASH）。富含新鲜蔬菜、水果、低脂（或
脱脂）乳制品、禽肉、鱼、大豆和坚果以及全谷物， 限制含
糖饮料和 红肉的摄入， 饱和脂肪酸和胆固醇水平低， 富含钾、
镁、钙等矿物质、优质蛋白质和膳食纤维。  
2.东方健康膳食模式。我国东南沿海地区居民高血压、
脑卒中的风险较低，期望寿命也较高，东南沿海一带的代表
性膳食统称为东方健康膳食模式。其主要特点是清淡少盐、
食物多样、谷物为主，蔬菜水果充足，鱼虾等水产品丰富，
奶类、豆类丰富等，并且具有较高的身体活动量。  
3.中国心脏健康膳食（ CHH-Diet）。与中国城市人群普
通膳食相比， 本膳食模式 减少钠摄入， 同时减少了脂肪 摄入，
增加了蛋白质、碳水化合物、钾、镁、钙和膳食纤维 摄入。


In [59]:
#向量检索
retriever = vectorstore.as_retriever()
docs = retriever.get_relevant_documents("东方健康膳食模式")
print(docs[0].page_content)

8 
 1.得舒饮食（ DASH）。富含新鲜蔬菜、水果、低脂（或
脱脂）乳制品、禽肉、鱼、大豆和坚果以及全谷物， 限制含
糖饮料和 红肉的摄入， 饱和脂肪酸和胆固醇水平低， 富含钾、
镁、钙等矿物质、优质蛋白质和膳食纤维。  
2.东方健康膳食模式。我国东南沿海地区居民高血压、
脑卒中的风险较低，期望寿命也较高，东南沿海一带的代表
性膳食统称为东方健康膳食模式。其主要特点是清淡少盐、
食物多样、谷物为主，蔬菜水果充足，鱼虾等水产品丰富，
奶类、豆类丰富等，并且具有较高的身体活动量。  
3.中国心脏健康膳食（ CHH-Diet）。与中国城市人群普
通膳食相比， 本膳食模式 减少钠摄入， 同时减少了脂肪 摄入，
增加了蛋白质、碳水化合物、钾、镁、钙和膳食纤维 摄入。


d:\Anaconda3\envs\pytorch\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [61]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system_template = "你是一名资深的医生，对于高血压有很深的经验"
human_template = "请根据以下文档回答我的问题：\n\n{docs}\n\n问题：{question}"

prompt = ChatPromptTemplate.from_messages([
        ("system", system_template),
        ("human" , human_template)
    ])

output_parser = StrOutputParser()

chain = prompt | llm | output_parser
question = "高血压患者应该怎么饮食？"
docs = retriever.get_relevant_documents(question)

chain.invoke({"question": question, "docs": docs})

' 高血压患者应该遵循合理膳食原则，平衡膳食，科学食养，少盐少糖，限制钠盐摄入，控制饮食热量等。同时，饮食要清淡，易消化，少食多餐，适当增加膳食纤维摄入，避免高脂肪、高胆固醇、高钠盐等食物。需要注意的是，饮食需根据个体差异而定，建议在医生或营养师的指导下进行饮食调整。<eod>'